## TP 08/01

In [1]:
# Choisir une seule catégorie dans train et entraîner le modèle
# Prétraiter les données txt + ann --> tokens
# Conversion ann en BIO
# Charger le modèle avec BERT, entraînement et prédiction

### Lire tous les fichiers test et train

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [4]:
# Chemin des dossiers test et train du dossier ANAT
anat_train_path = "C:/Users/SCD UM/OneDrive/Documents/Cours M2 2024-2025/S1/De la donnée de santé à l'IA/EMEA-20241220T094346Z-001/EMEA/train/ANAT"
anat_test_path = "C:/Users/SCD UM/OneDrive/Documents/Cours M2 2024-2025/S1/De la donnée de santé à l'IA/EMEA-20241220T094346Z-001/EMEA/test/ANAT"

for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith(".txt"): # Filtrer par extension
            file_path_txt = os.path.join(root, file)
            print(file_path_txt)

for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith(".ann"): # Filtrer par extension
            file_path_txt = os.path.join(root, file)
            print(file_path_txt)

C:/Users/SCD UM/OneDrive/Documents/Cours M2 2024-2025/S1/De la donnée de santé à l'IA/EMEA-20241220T094346Z-001/EMEA\test\ANAT\196_1.txt
C:/Users/SCD UM/OneDrive/Documents/Cours M2 2024-2025/S1/De la donnée de santé à l'IA/EMEA-20241220T094346Z-001/EMEA\test\ANAT\196_2.txt
C:/Users/SCD UM/OneDrive/Documents/Cours M2 2024-2025/S1/De la donnée de santé à l'IA/EMEA-20241220T094346Z-001/EMEA\test\ANAT\196_3.txt
C:/Users/SCD UM/OneDrive/Documents/Cours M2 2024-2025/S1/De la donnée de santé à l'IA/EMEA-20241220T094346Z-001/EMEA\test\ANAT\281_1.txt
C:/Users/SCD UM/OneDrive/Documents/Cours M2 2024-2025/S1/De la donnée de santé à l'IA/EMEA-20241220T094346Z-001/EMEA\test\ANAT\281_2.txt
C:/Users/SCD UM/OneDrive/Documents/Cours M2 2024-2025/S1/De la donnée de santé à l'IA/EMEA-20241220T094346Z-001/EMEA\test\ANAT\281_3.txt
C:/Users/SCD UM/OneDrive/Documents/Cours M2 2024-2025/S1/De la donnée de santé à l'IA/EMEA-20241220T094346Z-001/EMEA\test\ANAT\281_4.txt
C:/Users/SCD UM/OneDrive/Documents/Cours 

### Lire les fichiers test et train d'un dossier spécifique

In [5]:
# Chemin des dossiers test et train du dossier ANAT
anat_train_path = "C:/Users/SCD UM/OneDrive/Documents/Cours M2 2024-2025/S1/De la donnée de santé à l'IA/EMEA-20241220T094346Z-001/EMEA/train/ANAT"
anat_test_path = "C:/Users/SCD UM/OneDrive/Documents/Cours M2 2024-2025/S1/De la donnée de santé à l'IA/EMEA-20241220T094346Z-001/EMEA/test/ANAT"

# Fonction pour lire les fichiers .ann
def read_ann_files(path):
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".ann"):  # Filtrer par extension .ann
                file_path = os.path.join(root, file)
                print(f"Lecture du fichier : {file_path}")
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                    print(content)
                    print("-" * 50)  # Séparation entre les fichiers

# Lecture des fichiers ann de train
read_ann_files(anat_train_path)

# Lecture des fichiers ann de test
read_ann_files(anat_test_path)


Lecture du fichier : C:/Users/SCD UM/OneDrive/Documents/Cours M2 2024-2025/S1/De la donnée de santé à l'IA/EMEA-20241220T094346Z-001/EMEA/train/ANAT\318.ann
T18	ANAT 378 394	intrarachidienne
T20	ANAT 438 453	moelle épinière
T21	ANAT 438 444	moelle
T22	ANAT 460 467	cerveau
T27	ANAT 560 576	intrarachidienne
T30	ANAT 621 627	moelle
T44	ANAT 978 994	intrarachidienne
T49	ANAT 1098 1112	intrarachidien
T51	ANAT 1135 1150	canal rachidien
T52	ANAT 1135 1140	canal
T53	ANAT 1141 1150	rachidien
T78	ANAT 1970 1976	canaux
T80	ANAT 1992 1998	canaux
T81	ANAT 2028 2046	cellules nerveuses
T82	ANAT 2028 2036	cellules
T83	ANAT 2037 2046	nerveuses
T89	ANAT 2134 2152	cellules nerveuses
T90	ANAT 2134 2142	cellules
T91	ANAT 2143 2152	nerveuses
T96	ANAT 2231 2237	moelle
T124	ANAT 3003 3011	oculaire
T137	ANAT 3160 3164	cœur
T157	ANAT 3617 3633	intrarachidienne
T160	ANAT 3668 3683	canal rachidien
T161	ANAT 3668 3673	canal
T162	ANAT 3674 3683	rachidien
T168	ANAT 3845 3860	intrarachidiens
T197	ANAT 4936 4952	intra

### Prétraiter les données txt et ann

In [5]:
import os
import pandas as pd

# Fonction pour lire les fichiers .ann et regrouper dans une liste
def read_ann_files_to_dataframe(path):
    data = []  # Liste pour stocker les données
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".ann"):  # Filtrer par extension .ann
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                # Ajouter les données dans la liste sous forme de dictionnaire
                data.append({"file_name": file, "content": content})
    # Créer un DataFrame à partir de la liste
    return pd.DataFrame(data)

# Lire les fichiers .ann des dossiers train et test
df_train = read_ann_files_to_dataframe(anat_train_path)
df_test = read_ann_files_to_dataframe(anat_test_path)

# Fusionner les deux DataFrames 
df_all = pd.concat([df_train, df_test], ignore_index=True)

# Afficher le DataFrame final
print(df_all)


    file_name                                            content
0     318.ann  T18\tANAT 378 394\tintrarachidienne\nT20\tANAT...
1   345_1.ann  T11\tANAT 226 231\tveine\nT25\tANAT 567 587\tp...
2   345_2.ann  T4\tANAT 122 126\trein\nT12\tANAT 742 750\tcor...
3   345_3.ann  T87\tANAT 2155 2162\torganes\nT90\tANAT 2243 2...
4   425_1.ann  T7\tANAT 135 140\tveine\nT23\tANAT 487 502\tsy...
5   425_2.ann  T11\tANAT 396 402\trénale\nT13\tANAT 406 415\t...
6   425_3.ann  T23\tANAT 1102 1115\tlait maternel\nT24\tANAT ...
7   425_4.ann  T4\tANAT 123 132\tcérébrale\nT7\tANAT 231 254\...
8   425_5.ann                                                   
9   425_6.ann  T12\tANAT 335 339\tbras\nT13\tANAT 347 353\tja...
10  425_7.ann  T3\tANAT 44 54\thépatiques\nT5\tANAT 93 97\tpe...
11  196_1.ann  T24\tANAT 444 448\tpeau\nT26\tANAT 469 475\tth...
12  196_2.ann  T38\tANAT 1315 1319\tpeau\nT41\tANAT 1369 1374...
13  196_3.ann  T5\tANAT 264 268\tpeau\nT24\tANAT 706 712\tbou...
14  281_1.ann  T18\tANAT 

In [6]:
## Tokenisation

import os
import pandas as pd
import re

# Chemins des dossiers train et test
anat_train_path = "C:/Users/SCD UM/OneDrive/Documents/Cours M2 2024-2025/S1/De la donnée de santé à l'IA/EMEA-20241220T094346Z-001/EMEA/train/ANAT"
anat_test_path = "C:/Users/SCD UM/OneDrive/Documents/Cours M2 2024-2025/S1/De la donnée de santé à l'IA/EMEA-20241220T094346Z-001/EMEA/test/ANAT"

# Fonction pour extraire les tokens
def extract_tokens(df):
    """
    Extraire les tokens (mots) de chaque texte dans un DataFrame.

    Paramètres:
        df (pd.DataFrame): DataFrame contenant une colonne 'content' avec les textes.

    Retourne:
        list: Liste contenant les tokens extraits pour chaque texte.
    """
    all_tokens = []  # Liste pour stocker les tokens de chaque texte

    for text in df['content']:
        # Extraire les mots (tokens) avec un regex
        tokens = re.findall(r'\b\w+\b', text)
        all_tokens.append(tokens)

    return all_tokens


# Extraire les tokens pour les fichiers train et test
tokens_train = extract_tokens(df_train)

# Afficher les tokens pour les fichiers train
print("Tokens pour les fichiers TRAIN :")
for i, tokens in enumerate(tokens_train):
    print(f"Fichier {df_train.iloc[i]['file_name']} : {tokens}")
print("\n" + "=" * 80 + "\n")




Tokens pour les fichiers TRAIN :
Fichier 318.ann : ['T18', 'ANAT', '378', '394', 'intrarachidienne', 'T20', 'ANAT', '438', '453', 'moelle', 'épinière', 'T21', 'ANAT', '438', '444', 'moelle', 'T22', 'ANAT', '460', '467', 'cerveau', 'T27', 'ANAT', '560', '576', 'intrarachidienne', 'T30', 'ANAT', '621', '627', 'moelle', 'T44', 'ANAT', '978', '994', 'intrarachidienne', 'T49', 'ANAT', '1098', '1112', 'intrarachidien', 'T51', 'ANAT', '1135', '1150', 'canal', 'rachidien', 'T52', 'ANAT', '1135', '1140', 'canal', 'T53', 'ANAT', '1141', '1150', 'rachidien', 'T78', 'ANAT', '1970', '1976', 'canaux', 'T80', 'ANAT', '1992', '1998', 'canaux', 'T81', 'ANAT', '2028', '2046', 'cellules', 'nerveuses', 'T82', 'ANAT', '2028', '2036', 'cellules', 'T83', 'ANAT', '2037', '2046', 'nerveuses', 'T89', 'ANAT', '2134', '2152', 'cellules', 'nerveuses', 'T90', 'ANAT', '2134', '2142', 'cellules', 'T91', 'ANAT', '2143', '2152', 'nerveuses', 'T96', 'ANAT', '2231', '2237', 'moelle', 'T124', 'ANAT', '3003', '3011', 'ocul

In [18]:
## Nettoyage des données

In [15]:
# Installation de 'nltk' pour procéder au nettoyage des données

In [16]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [17]:
import pandas as pd
import numpy as np
import nltk
import re

In [ ]:
## Nettoyage des fichiers ann
import os
import pandas as pd
import re

class TextCleaner:
    @staticmethod
    def clean(text):
        """Nettoyage basique du texte"""
        if not isinstance(text, str):
            return ""
        
        # Conversion en minuscules
        text = text.lower()
        
        # Suppression des URLs
        text = re.sub(r'http\S+|www\S+', '', text)
        
        # Suppression des occurrences de 'removed'
        text = re.sub(r'removed', '', text)
        
        # Suppression des caractères spéciaux et des chiffres
        text = re.sub(r'[^\w\s]', ' ', text)
        
        # Normalisation des espaces
        text = re.sub(r'\s+', ' ', text).strip()
        
        return text
    
    @staticmethod
    def extract_words(text, min_length=3):
        """Extraction des mots avec longueur minimale"""
        cleaned = TextCleaner.clean(text)
        return [word for word in cleaned.split() if len(word) >= min_length]

    @staticmethod
    def extract_tokens(words):
        """Extraction des mots uniques (tokens)"""
        return list(set(words))

def read_and_clean_ann_files(path):
    """Lire les fichiers .ann, les nettoyer et les stocker dans un DataFrame"""
    data = []  # Liste pour stocker les données nettoyées
    
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".ann"):
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                
                # Nettoyage du texte
                cleaned_text = TextCleaner.clean(content)
                words = TextCleaner.extract_words(cleaned_text)
                tokens = TextCleaner.extract_tokens(words)
                
                # Ajouter les données nettoyées à la liste
                data.append({
                    'file_name': file,
                    'texte_nettoye': cleaned_text,
                    'mots': ' '.join(words),
                    'tokens': ' '.join(tokens),
                    'nombre_mots': len(words)
                })
    
    # Retourner un DataFrame contenant les données nettoyées
    return pd.DataFrame(data)

# Chemins des dossiers train 
anat_train_path = "C:/Users/SCD UM/OneDrive/Documents/Cours M2 2024-2025/S1/De la donnée de santé à l'IA/EMEA-20241220T094346Z-001/EMEA/train/ANAT"


# Nettoyage des fichiers .ann des répertoires train et test
df_train_cleaned = read_and_clean_ann_files(anat_train_path)

# Affichage des résultats
print("Données nettoyées pour TRAIN :")
print(df_train_cleaned)



Données nettoyées pour TRAIN :
    file_name                                      texte_nettoye  \
0     318.ann  t18 anat 378 394 intrarachidienne t20 anat 438...   
1   345_1.ann  t11 anat 226 231 veine t25 anat 567 587 plaque...   
2   345_2.ann  t4 anat 122 126 rein t12 anat 742 750 corporel...   
3   345_3.ann  t87 anat 2155 2162 organes t90 anat 2243 2249 ...   
4   425_1.ann  t7 anat 135 140 veine t23 anat 487 502 système...   
5   425_2.ann  t11 anat 396 402 rénale t13 anat 406 415 hépat...   
6   425_3.ann  t23 anat 1102 1115 lait maternel t24 anat 1102...   
7   425_4.ann  t4 anat 123 132 cérébrale t7 anat 231 254 syst...   
8   425_5.ann                                                      
9   425_6.ann  t12 anat 335 339 bras t13 anat 347 353 jambes ...   
10  425_7.ann  t3 anat 44 54 hépatiques t5 anat 93 97 peau t6...   

                                                 mots  \
0   t18 anat 378 394 intrarachidienne t20 anat 438...   
1   t11 anat 226 231 veine t25 anat 56

In [8]:
## Nettoyage des fichiers txt
import os
import pandas as pd
import re

class TextCleaner:
    @staticmethod
    def clean(text):
        """Nettoyage basique du texte"""
        if not isinstance(text, str):
            return ""
        
        # Conversion en minuscules
        text = text.lower()
        
        # Suppression des URLs
        text = re.sub(r'http\S+|www\S+', '', text)
        
        # Suppression des occurrences de 'removed'
        text = re.sub(r'removed', '', text)
        
        # Suppression des caractères spéciaux et des chiffres
        text = re.sub(r'[^\w\s]', ' ', text)
        
        # Normalisation des espaces
        text = re.sub(r'\s+', ' ', text).strip()
        
        return text
    
    @staticmethod
    def extract_words(text, min_length=3):
        """Extraction des mots avec longueur minimale"""
        cleaned = TextCleaner.clean(text)
        return [word for word in cleaned.split() if len(word) >= min_length]

    @staticmethod
    def extract_tokens(words):
        """Extraction des mots uniques (tokens)"""
        return list(set(words))

def read_and_clean_txt_files(path):
    """Lire les fichiers .txt, les nettoyer et les stocker dans un DataFrame"""
    data = []  # Liste pour stocker les données nettoyées
    
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".txt"):  # Filtrer les fichiers .txt
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                
                # Nettoyage du texte
                cleaned_text = TextCleaner.clean(content)
                words = TextCleaner.extract_words(cleaned_text)
                tokens = TextCleaner.extract_tokens(words)
                
                # Ajouter les données nettoyées à la liste
                data.append({
                    'file_name': file,
                    'texte_nettoye': cleaned_text,
                    'mots': ' '.join(words),
                    'tokens': ' '.join(tokens),
                    'nombre_mots': len(words)
                })
    
    # Retourner un DataFrame contenant les données nettoyées
    return pd.DataFrame(data)

# Chemins des dossiers train et test pour les fichiers .txt
anat_train_path = "C:/Users/SCD UM/OneDrive/Documents/Cours M2 2024-2025/S1/De la donnée de santé à l'IA/EMEA-20241220T094346Z-001/EMEA/train/ANAT"

# Nettoyage des fichiers .txt des répertoires train et test
df_train_cleaned = read_and_clean_txt_files(anat_train_path)


# Affichage des résultats
print("Données nettoyées pour TRAIN :")
print(df_train_cleaned)


Données nettoyées pour TRAIN :
    file_name                                      texte_nettoye  \
0     318.txt  emea h c 551 prialt qu est ce que prialt prial...   
1   345_1.txt  emea h c 122 refludan qu est ce que refludan r...   
2   345_2.txt  emea h c 122 recommandations standard comme la...   
3   345_3.txt  emea h c 122 4 3 contre indications csl behrin...   
4   425_1.txt  emea h c 603 tysabri qu est ce que tysabri tys...   
5   425_2.txt  tysabri 300 mg est administré en perfusion int...   
6   425_3.txt  l instauration d autres traitements au cours d...   
7   425_4.txt  l interaction entre l intégrine α 4β 1 et ses ...   
8   425_5.txt  solution diluée à conserver au réfrigérateur e...   
9   425_6.txt  qu est ce que tysabri et dans quel cas est il ...   
10  425_7.txt  signes évocateurs d éventuels troubles hépatiq...   

                                                 mots  \
0   emea 551 prialt est que prialt prialt est une ...   
1   emea 122 refludan est que refludan

### Conversion des fichiers ann en BIO

In [16]:
import os
import re

def read_txt_file(txt_file_path):
    """Lire le contenu du fichier .txt"""
    with open(txt_file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return content

def read_ann_file(ann_file_path):
    """Lire le contenu du fichier .ann"""
    with open(ann_file_path, 'r', encoding='utf-8') as file:
        content = file.read().splitlines()
    return content

def convert_to_bio(text, annotations):
    """Convertir le texte et les annotations en format BIO"""
    # Tokeniser le texte en mots
    words = re.findall(r'\b\w+\b', text)
    
    # Initialiser les tags BIO avec 'O' pour chaque mot
    bio_tags = ['O'] * len(words)
    
    # Extraire les entités et leurs positions depuis les annotations
    named_entities = []
    for annotation in annotations:
        parts = annotation.strip().split("\t")
        if len(parts) < 3:
            continue
        entity_type = parts[1]  # Nom de l'entité
        entity_span = parts[2]  # Position de l'entité (ex : '0 4')
        
        try:
            # Vérifier si les positions de l'entité sont des entiers valides
            entity_start, entity_end = map(int, entity_span.split())
            # Ajouter l'entité à la liste des entités nommées
            named_entities.append((entity_start, entity_end, entity_type))
        except ValueError:
            # Si la conversion échoue, ignorer cette annotation
            print(f"Annotation mal formatée (position invalide) : {annotation}")
            continue
    
    # Appliquer les tags BIO pour chaque mot
    for i, word in enumerate(words):
        for start, end, entity_type in named_entities:
            # Vérifier si le mot fait partie d'une entité
            if start <= i < end:
                if bio_tags[i] == 'O':  # Si le mot n'a pas encore de tag
                    bio_tags[i] = 'B-' + entity_type  # B pour le début de l'entité
                else:
                    bio_tags[i] = 'I-' + entity_type  # I pour l'intérieur de l'entité

    return ' '.join(bio_tags)

def process_files_in_directory(directory_path):
    """Traiter tous les fichiers .txt et .ann dans un dossier donné"""
    bio_data = []
    
    # Parcours des fichiers du dossier
    for root, dirs, files in os.walk(directory_path):
        txt_files = [f for f in files if f.endswith(".txt")]
        
        for txt_file in txt_files:
            txt_file_path = os.path.join(root, txt_file)
            ann_file_path = os.path.join(root, txt_file.replace(".txt", ".ann"))
            
            # Vérifier si le fichier .ann existe
            if os.path.exists(ann_file_path):
                text = read_txt_file(txt_file_path)  # Lire le texte
                annotations = read_ann_file(ann_file_path)  # Lire les annotations
                
                # Convertir en format BIO
                bio_tags = convert_to_bio(text, annotations)
                bio_data.append(bio_tags)  # Ajouter les tags BIO à la liste
    
    return bio_data

# Exemple de traitement des fichiers d'entraînement et de test
anat_train_path = "C:/Users/SCD UM/OneDrive/Documents/Cours M2 2024-2025/S1/De la donnée de santé à l'IA/EMEA-20241220T094346Z-001/EMEA/train/ANAT"  # Remplacer par le bon chemin
anat_test_path = "C:/Users/SCD UM/OneDrive/Documents/Cours M2 2024-2025/S1/De la donnée de santé à l'IA/EMEA-20241220T094346Z-001/EMEA/test/ANAT"  # Remplacer par le bon chemin

bio_data_train = process_files_in_directory(anat_train_path)
bio_data_test = process_files_in_directory(anat_test_path)

# Afficher les résultats pour vérifier
print("BIO tags pour les données d'entraînement :")
for bio in bio_data_train:
    print(bio)

print("\nBIO tags pour les données de test :")
for bio in bio_data_test:
    print(bio)


Annotation mal formatée (position invalide) : T18	ANAT 378 394	intrarachidienne
Annotation mal formatée (position invalide) : T20	ANAT 438 453	moelle épinière
Annotation mal formatée (position invalide) : T21	ANAT 438 444	moelle
Annotation mal formatée (position invalide) : T22	ANAT 460 467	cerveau
Annotation mal formatée (position invalide) : T27	ANAT 560 576	intrarachidienne
Annotation mal formatée (position invalide) : T30	ANAT 621 627	moelle
Annotation mal formatée (position invalide) : T44	ANAT 978 994	intrarachidienne
Annotation mal formatée (position invalide) : T49	ANAT 1098 1112	intrarachidien
Annotation mal formatée (position invalide) : T51	ANAT 1135 1150	canal rachidien
Annotation mal formatée (position invalide) : T52	ANAT 1135 1140	canal
Annotation mal formatée (position invalide) : T53	ANAT 1141 1150	rachidien
Annotation mal formatée (position invalide) : T78	ANAT 1970 1976	canaux
Annotation mal formatée (position invalide) : T80	ANAT 1992 1998	canaux
Annotation mal form

In [17]:
import os
import re

# Fonction pour lire un fichier .txt
def read_txt_file(txt_file_path):
    with open(txt_file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return content

# Fonction pour lire un fichier .ann
def read_ann_file(ann_file_path):
    with open(ann_file_path, 'r', encoding='utf-8') as file:
        content = file.read().splitlines()
    return content

# Fonction pour tokeniser le texte (nltk ou re)
def tokenize_text(text):
    # Utilisation de re.findall pour obtenir les mots comme tokens
    return re.findall(r'\b\w+\b', text)

# Fonction pour convertir les annotations en format BIO
def convert_to_bio(text, annotations):
    # Tokeniser le texte
    tokens = tokenize_text(text)
    
    # Créer une liste de tags BIO, initialisée à "O" pour chaque token
    bio_tags = ['O'] * len(tokens)
    
    # Convertir les annotations en format utilisable
    entities = []
    for annotation in annotations:
        parts = annotation.strip().split("\t")
        if len(parts) < 3:
            continue  # Ignorer les annotations mal formatées

        entity_type = parts[1]  # Type d'entité (par ex. "Disease")
        entity_span = parts[2]  # Position de l'entité, par ex. "0 3"
        
        try:
            # Obtenir les positions de début et de fin de l'entité
            entity_start, entity_end = map(int, entity_span.split())
            
            # Associer l'entité à une plage de tokens
            entities.append((entity_start, entity_end, entity_type))
        except ValueError:
            continue  # Ignorer les annotations mal formatées

    # Appliquer les tags BIO aux tokens
    for i, token in enumerate(tokens):
        for start, end, entity_type in entities:
            # Vérifier si le token fait partie de l'entité
            if start <= i < end:
                if bio_tags[i] == 'O':  # Si le token n'a pas encore de tag
                    bio_tags[i] = 'B-' + entity_type  # Marquer le début de l'entité
                else:
                    bio_tags[i] = 'I-' + entity_type  # Marquer l'intérieur de l'entité
    
    # Retourner les tags BIO sous forme de chaîne
    return ' '.join(bio_tags)

# Fonction pour traiter tous les fichiers d'un dossier
def process_files_in_directory(directory_path):
    bio_data = []
    
    for root, dirs, files in os.walk(directory_path):
        txt_files = [f for f in files if f.endswith(".txt")]
        
        for txt_file in txt_files:
            txt_file_path = os.path.join(root, txt_file)
            ann_file_path = os.path.join(root, txt_file.replace(".txt", ".ann"))
            
            # Vérifier si le fichier .ann existe
            if os.path.exists(ann_file_path):
                text = read_txt_file(txt_file_path)  # Lire le texte
                annotations = read_ann_file(ann_file_path)  # Lire les annotations
                
                # Convertir en format BIO
                bio_tags = convert_to_bio(text, annotations)
                bio_data.append(bio_tags)  # Ajouter les tags BIO à la liste
    
    return bio_data

# Exemple d'utilisation pour les données d'entraînement et de test
anat_train_path = "C:/Users/SCD UM/OneDrive/Documents/Cours M2 2024-2025/S1/De la donnée de santé à l'IA/EMEA-20241220T094346Z-001/EMEA/train/ANAT"  # Remplacer par le bon chemin
anat_test_path = "C:/Users/SCD UM/OneDrive/Documents/Cours M2 2024-2025/S1/De la donnée de santé à l'IA/EMEA-20241220T094346Z-001/EMEA/test/ANAT"  # Remplacer par le bon chemin

bio_data_train = process_files_in_directory(anat_train_path)
bio_data_test = process_files_in_directory(anat_test_path)

# Afficher les résultats pour vérifier
print("BIO tags pour les données d'entraînement :")
for bio in bio_data_train:
    print(bio)

print("\nBIO tags pour les données de test :")
for bio in bio_data_test:
    print(bio)


BIO tags pour les données d'entraînement :
O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O

In [ ]:
df.head().apply(get_iob_format_from_row,axis=1)

In [ ]:
# iob with sentiment info
df.head().apply(lambda x:get_iob_format_from_row(x, add_sentiment=True), axis=1)

### Entraînement et prédiction avec BERT